# Heuristics

This notebook explores heuristic and local search methods for the One-Sided Cross Minimization problem.

In [ ]:
import sys
import os
import random
import copy
import math
from typing import TypeVar, TextIO, Tuple, List, Dict
import segment_tree as sg
from scipy.optimize import linear_sum_assignment
import time
import statistics
import pickle

random.seed(int(5787549218))

PATH = "./data" # data directory
GRAPH = os.path.join(PATH, "heuristic/90.gr") # singular graph example

In [ ]:
# Reference: pace2024-verifier package (https://pypi.org/project/pace2024-verifier/)
class PaceGraph():
    # Only the right side is flexible
    left: int
    right: list
    right_order: Dict[int, int]
    edgeset: List[Tuple[int, int]]
    edges: Dict[int, List[int]] # adjacency list

    # Build a graph with sides A and B of size a and b. The side B is assumed to be flexible and A to be fixed.
    # If order is given we set the right side to be ordered that way. 
    def __init__(self, a: int, b: int, edges: list, order: list = None):
        if order:
            oelements = set(order)
            relements = set(range(a + 1, a + b + 1))
            diff = oelements - relements
            if len(diff) != 0 :
                raise ValueError(f"Error! 'order' contains elements not present in the graph {diff}.")
            else:
                self.right = order
        else:
            self.right = list(range(a + 1, a + b + 1))
        
        self.right_order = {u: p for p, u in enumerate(self.right)} # {vertex_id: position}
        self.left = a
        self.edges = {}
        self.edgeset = []
        edges = sorted(edges) # sort all edges such that we get sorted adjacency lists
        for e in edges:
            u = e[0] if e[0] < e[1] else e[1]
            v = e[0] if e[1] < e[0] else e[1]
            if not (u in self.edges.keys()):
                self.edges[u] = []
            if not (v in self.edges.keys()):
                self.edges[v] = []
            self.edges[u].append(v)
            self.edges[v].append(u)
            self.edgeset.append((u,v))
        for v in range(1,a+b+1):
            if v not in self.edges.keys():
                self.edges[v] = []

    # Return the neighbors of the given vertex
    def neighbors(self, u: int) -> int:
        return self.edges[u]
    
    # Set the order to the given list
    def set_order(self, order: List[int], check: bool = False):
        if check:
            oelements = set(order)
            relements = set(range(self.left + 1, len(self.right) + self.left + 1))
            diff = oelements - relements
            if len(diff) != 0 :
                raise ValueError(f"Error! 'order' contains elements not present in the graph {diff}.")
            else:
                self.right = order
        self.right = order
        self.right_order = {u: p for p, u in enumerate(self.right)}

    # Check if edges (a,b) and (c,d) cross
    def cross(self, a: int, b: int, c: int, d: int) -> bool:
        if a > b:
            a, b = b, a
        if c > d:
            c, d = d, c
        b = self.right_order[b]
        d = self.right_order[d]
        return (a < c and b > d) or (c < a and d > b)
    
    # Swap the position of two vertices in the order
    def swap(self, a: int, b: int):
        apos = self.right_order[a]
        bpos = self.right_order[b]
        self.right[apos], self.right[bpos] = self.right[bpos], self.right[apos]
        self.right_order[a] = bpos
        self.right_order[b] = apos

    
    def countcrossings_segtree(self) -> int:
        size = len(self.right)
        edges = sorted(self.edgeset, key=lambda e: (e[0], self.right_order[e[1]]))
        arr = [0] * (size + 1)
        t = sg.SegmentTree(arr)
        crossings = 0
        for edge in edges:
            old = t.query(self.right_order[edge[1]], self.right_order[edge[1]], "sum")
            t.update(self.right_order[edge[1]], old+1)
            if size > self.right_order[edge[1]]+1:
                crossings_found = t.query(self.right_order[edge[1]]+1, size, "sum")
                crossings += crossings_found
        return crossings


    def show(self, title=""):
        e = [(x, y) for (v, w) in self.edgeset 
                 if (x := self.right_order[v]+self.left+1 if v > self.left else v) and 
                    (y := self.right_order[w]+self.left+1 if w > self.left else w)
            ]
        g = BipartiteGraph(e)
        g.left = [i for i in range(1,self.left+1)]
        g.right = self.right
        return g.plot(title=title, vertex_labels=all(i < j for i, j in zip(self.right, self.right[1:])))

    
    def reassign_positions(self, assignment: List[Tuple[int, int]], check: bool = False): 
        # (a, b) means a should go where b is.
        if check:
            if set(x[0] for x in assignment) != set(x[1] for x in assignment):
                raise ValueError("Error! New assignment is not valid.")
        new_positions = [(old, self.right_order[new]) for old, new in assignment]
        for vertex, position in new_positions:
            self.right[position] = vertex
            self.right_order[vertex] = position

    
    def randomly_permute(self):
        random.shuffle(self.right)
        self.right_order = {u: p for p, u in enumerate(self.right)}


    def barycenter(self):
        x = [(v, sum(self.edges[v]) / max(1, len(self.edges[v]))) for v in self.right]
        x.sort(key=lambda x: x[1])
        new_order = [v[0] for v in x]
        self.set_order(new_order)

    
    def median(self):
        x = [[] for _ in range(self.left + 1)]
        for w in self.right:
            nbrs = self.edges[w]
            median = int(statistics.median(self.edges[w])) if nbrs else 0
            x[median].append(w)
        new_order = sum(x, [])
        self.set_order(new_order)
        

    def bipartite_distance(self, left_vertex: int, right_vertex: int):
        return abs(left_vertex - self.right_order[right_vertex] + 1)
    

In [ ]:
def load_graph(path):
    N0 = -1; N1 = -1
    E = list()
    with open(path, 'r') as f:
        for l in f.readlines():
            l = l.split()
            if l[0][0] == 'p':
                N0 = Integer(l[2])
                N1 = Integer(l[3])
            elif l[0][0] != 'c':
                edge = (Integer(l[0]), Integer(l[1]))
                E.append(edge)
    return PaceGraph(N0, N1, E)

G = load_graph(GRAPH)
print(f"{G.left} Left-Vertices, {len(G.right)} Right-Vertices, {len(G.edgeset)} Edges, {G.countcrossings_segtree()} Crossings.")

# Well-Known Heuristics

We compare the barycenter and median heuristics, which give $O(\sqrt{n})$ and $3$ factor approximations respectively

In [ ]:
G.barycenter()
print(f"Barycenter Heuristic Crossings: {G.countcrossings_segtree()}")

G.median()
print(f"Median Heuristic Crossings: {G.countcrossings_segtree()}")

# Iterative Heuristics

## Random Walk

We move through adjacent states, tracking the best state as we go. We obtain an adjacent state by swapping two vertices in the flexible layer of the graph.

In [ ]:
def random_walk(G, n_steps):
    min_G = copy.deepcopy(G)
    min_crossings = G.countcrossings_segtree()
    
    for _ in range(n_steps):
        # Get two random edges that cross and swap their right-layer endpoints.
        (a, b), (c, d) = random.sample(G.edgeset, k=2) 
        while not G.cross(a, b, c, d):
            (a, b), (c, d) = random.sample(G.edgeset, k=2)
        G.swap(b, d)
        c = G.countcrossings_segtree()
        if c < min_crossings:
            min_crossings = c
            min_G = copy.deepcopy(G)
    
    return min_G, min_crossings

In [ ]:
G = load_graph(GRAPH)
G.barycenter()
G, c = random_walk(G, 100)
print(f"{c} Crossings.")

## (Stochastic) Hill Climbing

Because the evaluation of each neighbour is costly, and the neighbourhood is large ($O(n^2)$), we use the stochastic variant of the general hill-climbing algorithm. That is, we move to a better state when one is found.

In [ ]:
def hill_climbing(G, n_steps, neighborhood_size):
    min_G = copy.deepcopy(G)
    min_crossings = G.countcrossings_segtree()
    record = []
    
    for _ in range(n_steps):
        # Get two random edges that cross and swap their right-layer endpoints.
        next_states = dict()
        while len(next_states) < neighborhood_size:
            (a, b), (c, d) = random.sample(G.edgeset, k=2) 
            while not G.cross(a, b, c, d):
                (a, b), (c, d) = random.sample(G.edgeset, k=2)
            G.swap(b, d)
            c = G.countcrossings_segtree()
            next_states[c] = (b, d)
            G.swap(b, d)
        c = min(next_states.keys())
        b, d = next_states[c]

        if c < min_crossings:
            min_crossings = c
            G.swap(b, d)
            min_G = copy.deepcopy(G)
        record.append(min_crossings)

    assert min_G.countcrossings_segtree() == min_crossings
    return min_G, min_crossings, record

In [ ]:
G = load_graph(GRAPH)
G.barycenter()
G, c, r = hill_climbing(G, 440, 10)
print(f"Minimum: {c} Crossings.")

## Simulated Annealing

We only accept better states according to a probability controlled by a temperature schedule:

```
Note that: delta = current_valuation - best_valuation

def accept(delta, temperature):
    if delta < 0 or random.random() < math.exp(-delta / temperature):
        return True
    else:
        return False
```

In [ ]:
def simulated_annealing(G, initial_temperature, cooling_rate, neighborhood_size=1):
    t = initial_temperature
    min_G = copy.deepcopy(G)
    min_crossings = G.countcrossings_segtree()
    record = []
    
    while t > 1:
        # Get two random edges that cross and swap their right-layer endpoints.
        next_states = dict()
        while len(next_states) < neighborhood_size:
            (a, b), (c, d) = random.sample(G.edgeset, k=2) 
            while not G.cross(a, b, c, d):
                (a, b), (c, d) = random.sample(G.edgeset, k=2)
            G.swap(b, d)
            c = G.countcrossings_segtree()
            next_states[c] = (b, d)
            G.swap(b, d)
        c = min(next_states.keys())
        b, d = next_states[c]
        
        if c < min_crossings:
            min_crossings = c
            G.swap(b, d)
            min_G = copy.deepcopy(G)
        elif random.random() < math.exp(-(c - min_crossings) / t):
            G.swap(b, d)
        
        record.append(min_crossings)
        t *= cooling_rate

    assert min_G.countcrossings_segtree() == min_crossings
    return min_G, min_crossings, record

In [ ]:
G = load_graph(GRAPH)
G.barycenter()
G, c, r = simulated_annealing(G, 100, 0.9, 10)
print(f"{c} Crossings.")

## Iterated Simulated Annealing

In [ ]:
def iterated_simulated_annealing(G, n_iterations, initial_temperature, cooling_rate, neighborhood_size=1):
    min_G = copy.deepcopy(G)
    min_crossings = G.countcrossings_segtree()
    record = []
    for _ in range(n_iterations):
        H = copy.deepcopy(min_G)
        H, c, r = simulated_annealing(H, initial_temperature, cooling_rate, neighborhood_size)
        #print(c)
        if c < min_crossings:
            min_crossings = c
            min_G = copy.deepcopy(H)
        record += r

    assert min_G.countcrossings_segtree() == min_crossings
    return min_G, min_crossings, record

In [ ]:
G = load_graph(GRAPH)
G.barycenter()
G, c, r = iterated_simulated_annealing(G, 10, 100, 0.9, 10)
print(f"{c} Crossings.")

## Introducing $k$-opt

So far we have been obtaining new solutions by uncrossing two edges. Doing so can be considered a 'local' optimisation of some kind, but it should be noted this makes no guarantees that this optimisation brings us closer to the global optimum. We may call the set of feasible solutions that we can reach by swapping two edges as the $2$-opt neighbourhood. We may examine the results of swapping $k$-edges in a similar way, i.e. swapping $k$ edges such that they have the least number of crossings with each other. We call this the $k$-opt neighbourhood.

In [ ]:
def k_opt_random_walk(G, k, n_steps):
    record = []
    for _ in range(n_steps):
        k_edges = [None for _ in range(k)]
        repeat = True
        s = time.time()
        while repeat and time.time() - s < 1:
            k_edges[0] = random.sample(G.edgeset, k=1)[0]
            endpoints = set(k_edges[0])
            for i in range(1, k):
                k_edges[i] = random.sample(G.edgeset, k=1)[0]
                while ((not any(G.cross(*edge, *k_edges[i]) for edge in k_edges[:i])) or 
                      (k_edges[i][0] in endpoints) or 
                      (k_edges[i][1] in endpoints)) and (time.time() - s < 1):
                    k_edges[i] = random.sample(G.edgeset, k=1)[0]
                endpoints.add(k_edges[i][0])
                endpoints.add(k_edges[i][1])
            if len(endpoints) == k*2:
                repeat = False

        # Create a "cost" matrix by 'distance'
        cost = [[None for _ in range(k)] for __ in range(k)]
        for i in range(k):
            for j in range(k):
                cost[i][j] = G.bipartite_distance(k_edges[i][0], k_edges[j][1])
        # Get the minimum-cost reassignment of right endpoints.
        row_assn, col_assn = linear_sum_assignment(cost, maximize=False)
        swaps = [(k_edges[old][1], k_edges[new][1]) for old, new in zip(row_assn, col_assn)]
        G.reassign_positions(swaps)
        c = G.countcrossings_segtree()
        record.append(c)
    
    return G, record

In [ ]:
for i in range(2,6):
    G = load_graph(GRAPH)
    #G.barycenter()
    G, c, r = k_opt_random_walk(G, i, 440)
    print(f"{c} Crossings, when k = {i}.")

## $k$-opt Hill Climbing

In [ ]:
def k_opt_hill_climbing(G, k, n_iterations, neighborhood_size=1):
    min_G = copy.deepcopy(G)
    min_crossings = G.countcrossings_segtree()
    record = []
    
    for _ in range(n_iterations):
        new_states = dict()
        while len(new_states) < neighborhood_size:
            k_edges = [None for _ in range(k)]
            repeat = True
            s = time.time()
            while repeat and time.time() - s < 1:
                k_edges[0] = random.sample(G.edgeset, k=1)[0]
                endpoints = set(k_edges[0])
                for i in range(1, k):
                    k_edges[i] = random.sample(G.edgeset, k=1)[0]
                    while ((not any(G.cross(*edge, *k_edges[i]) for edge in k_edges[:i])) or 
                          (k_edges[i][0] in endpoints) or 
                          (k_edges[i][1] in endpoints)) and (time.time() - s < 1):
                        k_edges[i] = random.sample(G.edgeset, k=1)[0]
                    endpoints.add(k_edges[i][0])
                    endpoints.add(k_edges[i][1])
                if len(endpoints) == k*2:
                    repeat = False
            if repeat:
                continue
            # Create a "cost" matrix by 'distance'
            cost = [[None for _ in range(k)] for __ in range(k)]
            for i in range(k):
                for j in range(k):
                    cost[i][j] = G.bipartite_distance(k_edges[i][0], k_edges[j][1])
            # Get the minimum-cost reassignment of right endpoints.
            row_assn, col_assn = linear_sum_assignment(cost, maximize=False)
            swaps = [(k_edges[old][1], k_edges[new][1]) for old, new in zip(row_assn, col_assn)]
            G.reassign_positions(swaps)
            c = G.countcrossings_segtree()
            new_states[c] = swaps
            G.reassign_positions([(x[1], x[0]) for x in swaps])

        c = min(new_states.keys())
        swaps = new_states[c]
        if c < min_crossings:
            min_crossings = c
            G.reassign_positions(swaps)
            min_G = copy.deepcopy(G)
        record.append(min_crossings)

    assert min_G.countcrossings_segtree() == min_crossings
    return min_G, min_crossings, record

In [ ]:
for i in range(2,6):
    G = load_graph(GRAPH)
    #G.barycenter()
    G, c, r = k_opt_hill_climbing(G, i, 440, 10)
    print(f"{c} Crossings, when k = {i}.")

## $k$-opt Simulated Annealing

In [ ]:
def k_opt_simulated_annealing(G, k, initial_temperature, cooling_rate, neighborhood_size=1):
    t = initial_temperature
    min_G = copy.deepcopy(G)
    min_crossings = G.countcrossings_segtree()
    record = []
    while t > 1:
        new_states = dict()
        while len(new_states) < neighborhood_size:
            k_edges = [None for _ in range(k)]
            repeat = True
            s = time.time()
            while repeat and time.time() - s < 1:
                k_edges[0] = random.sample(G.edgeset, k=1)[0]
                endpoints = set(k_edges[0])
                for i in range(1, k):
                    k_edges[i] = random.sample(G.edgeset, k=1)[0]
                    while ((not any(G.cross(*edge, *k_edges[i]) for edge in k_edges[:i])) or 
                          (k_edges[i][0] in endpoints) or 
                          (k_edges[i][1] in endpoints)) and (time.time() - s < 1):
                        k_edges[i] = random.sample(G.edgeset, k=1)[0]
                    endpoints.add(k_edges[i][0])
                    endpoints.add(k_edges[i][1])
                if len(endpoints) == k*2:
                    repeat = False
            if repeat:
                continue
    
            # Create a "cost" matrix by 'distance'
            cost = [[None for _ in range(k)] for __ in range(k)]
            for i in range(k):
                for j in range(k):
                    cost[i][j] = G.bipartite_distance(k_edges[i][0], k_edges[j][1])
            # Get the minimum-cost reassignment of right endpoints.
            row_assn, col_assn = linear_sum_assignment(cost, maximize=False)
            swaps = [(k_edges[old][1], k_edges[new][1]) for old, new in zip(row_assn, col_assn)]
            G.reassign_positions(swaps)
            c = G.countcrossings_segtree()
            new_states[c] = swaps
            G.reassign_positions([(x[1], x[0]) for x in swaps])

        c = min(new_states.keys())
        swaps = new_states[c]
        if c < min_crossings:
            min_crossings = c
            G.reassign_positions(swaps)
            min_G = copy.deepcopy(G)
        elif random.random() < math.exp(-(c - min_crossings) / t):
            G.reassign_positions(swaps)
        
        record.append(min_crossings)
        t *= cooling_rate

    assert min_G.countcrossings_segtree() == min_crossings
    return min_G, min_crossings, record

In [ ]:
for i in range(2,6):
    G = load_graph(GRAPH)
    #G.barycenter()
    G, c, r = k_opt_simulated_annealing(G, i, 100, 0.9, 10)
    print(f"{c} Crossings, when k = {i}.")

## Iterated $k$-opt Simulated Annealing

In [ ]:
def iterated_k_opt_simulated_annealing(G, n_iterations, k, initial_temperature, cooling_rate, neighborhood_size=1):
    min_G = copy.deepcopy(G)
    min_crossings = G.countcrossings_segtree()
    record = []
    for _ in range(n_iterations):
        H = copy.deepcopy(min_G)
        H, c, r = k_opt_simulated_annealing(H, k, initial_temperature, cooling_rate, neighborhood_size)
        #print(c)
        if c < min_crossings:
            min_crossings = c
            min_G = copy.deepcopy(H)
        record += r

    assert min_G.countcrossings_segtree() == min_crossings
    return min_G, min_crossings, record

In [ ]:
for i in range(2,6):
    G = load_graph(GRAPH)
    #G.barycenter()
    G, c, r = iterated_k_opt_simulated_annealing(G, 10, i, 100, 0.9, 10)
    print(f"{c} Crossings, when k = {i}.")

# Evaluations

We have two kinds of evaluations:
1. Intended for improving the barycenter heuristic.
2. From random permutation.

## Starting from the Barycenter Heuristic

Here we evaluate stochastic hill climbing, simulated annealing, and staged simulated annealing. We consider a fixed-sized neighbourhood of 10.

- Stochastic Hill Climbing: 440 iterations
- Simulated Annealing: 10 trials, 100 temp, 0.99 cooling rate
- Staged Simulated Annealing: 10 stages, 100 temp, 0.9 cooling rate

In [ ]:
tests = sorted([f for f in os.listdir(os.path.join(PATH, "chosen")) if os.path.isfile(os.path.join(os.path.join(PATH, "chosen"), f))])
results = {k:dict() for k in tests}

for test in tests:
    # Hill Climbing
    print(f"Test: {test}, HC")
    G = load_graph(os.path.join(PATH, f"chosen/{test}"))
    G.barycenter()
    G, c, results[test]["hill_climbing"] = hill_climbing(G, 440, 10)

    # Simulated Annealing
    print(f"Test: {test}, SA")
    results[test]["simulated_annealing"] = []
    #for i in range(10):
    G = load_graph(os.path.join(PATH, f"chosen/{test}"))
    G.barycenter()
    G, c, results[test]["simulated_annealing"] = simulated_annealing(G, 100, 0.98957, 10)
    
    # Staged S.A.
    print(f"Test: {test}, SSA")
    G = load_graph(os.path.join(PATH, f"chosen/{test}"))
    G.barycenter()
    G, c, results[test]["staged_simulated_annealing"] = iterated_simulated_annealing(G, 10, 100, 0.9, 10)

with open('./results/barycenter_results_NEWSSA.pkl', 'wb') as f:
    pickle.dump(results, f)

## Starting from a Random Permutation

Here we evaluate stochastic hill climbing and staged simulated annealing for various $k$.

In [ ]:
tests = ['22.gr', '79.gr', 'ang-wikibooks.gr']
results = {test:{"hill_climbing":dict(), 
                 "simulated_annealing":dict(), 
                 "staged_simulated_annealing":dict(),
                 "random_walk":dict()} for test in tests}

for i in [2, 3, 5, 10, 25, 50]:
    for test in tests:
        # Hill Climbing
        print(f"Test: {test}, k={i}, HC")
        G = load_graph(os.path.join(PATH, f"chosen/{test}"))
        G.randomly_permute()
        G, c, results[test]["hill_climbing"][i] = k_opt_hill_climbing(G, i, 440, 10)

        # S.A.
        print(f"Test: {test}, k={i}, SA")
        G = load_graph(os.path.join(PATH, f"chosen/{test}"))
        G.randomly_permute()
        G, c, results[test]["simulated_annealing"][i] = k_opt_simulated_annealing(G, i, 100, 0.98957, 10)
    
        # Staged S.A.
        print(f"Test: {test}, k={i}, SSA")
        G = load_graph(os.path.join(PATH, f"chosen/{test}"))
        G.randomly_permute()
        G, c, results[test]["staged_simulated_annealing"][i] = iterated_k_opt_simulated_annealing(G, 10, i, 100, 0.9, 10)

        # Random Walk
        print(f"Test: {test}, k={i}, RW")
        G = load_graph(os.path.join(PATH, f"chosen/{test}"))
        G.randomly_permute()
        G, results[test]["random_walk"][i] = k_opt_random_walk(G, i, 440)
    
with open('./results/kopt_results.pkl', 'wb') as f:
    pickle.dump(results, f)

End of notebook.

---


# Previous Work

The following contains previous attempts.

In [ ]:
from itertools import permutations

class KOptSubGraph():

    def __init__(self, edge_list: List[Tuple[int,int]]):
        # Each edge must be given as (left, right).
        # The ordering of vertices in each layer is in order of their appearances in the list.
        self.left = []
        self.right = []
        self.right_order = {}
        self.neighbors = {}
        self.vertices = set()
        for i, edge in enumerate(edge_list):
            if edge[0] not in self.vertices:
                self.left.append(edge[0])
                self.vertices.add(edge[0])
            if edge[1] not in self.vertices:
                self.right.append(edge[1])
                self.right_order[edge[1]] = i
                self.vertices.add(edge[1])
            if edge[0] not in self.neighbors.keys():
                self.neighbors[edge[0]] = []
            if edge[1] not in self.neighbors.keys():
                self.neighbors[edge[1]] = []
            self.neighbors[edge[0]].append(edge[1])
            self.neighbors[edge[1]].append(edge[0])

    def set_order(self, order: List[int]):
        self.right = order
        self.right_order = {u: p for p, u in enumerate(self.right)}

    def cross(self, a: int, b: int, c: int, d: int) -> bool:
        if a > b:
            a, b = b, a
        if c > d:
            c, d = d, c
        b = self.right_order[b]
        d = self.right_order[d]
        return (a < c and b > d) or (c < a and d > b)

    def count_crossings(self) -> int:
        crossings = 0
        for i1 in range(len(self.left)-1):
            for i2 in range(i1 + 1, len(self.left)):
                for w1 in self.neighbors[self.left[i1]]:
                    for w2 in self.neighbors[self.left[i2]]:
                        if self.cross(self.left[i1], w1, self.left[i2], w2):
                            crossings += 1
        return int(crossings)


def k_opt_hill_climbing(G, k, n_iterations):
    min_G = copy.deepcopy(G)
    min_crossings = G.countcrossings_segtree()
    
    for _ in range(n_iterations):
        # Get k random edges s.t. every edge crosses at least one of the other k-1 edges
        #print(_, end=" ")
        k_edges = [None for _ in range(k)]

        repeat = True
        s = time.time()
        while repeat and (time.time() - s < 1):
            k_edges[0] = random.sample(G.edgeset, k=1)[0]
            endpoints = set([k_edges[0][1]])
            for i in range(1, k):
                k_edges[i] = random.sample(G.edgeset, k=1)[0]
                while (not any(G.cross(*edge, *k_edges[i]) for edge in k_edges[:i]) and
                       time.time() - s < 1):
                    k_edges[i] = random.sample(G.edgeset, k=1)[0]
                endpoints.add(k_edges[i][1])
            if len(endpoints) < 2:
                continue
            suborder = list(endpoints)
            suborder.sort(key=lambda x: G.right_order[x])
            kosg = KOptSubGraph(k_edges)
            kosg.set_order(suborder)
            min_kopt_c = kosg.count_crossings()
            min_kopt_o = suborder
            for perm in permutations(suborder):
                kosg.set_order(perm)
                kopt_c = kosg.count_crossings()
                if kopt_c < min_kopt_c:
                    min_kopt_c = kopt_c
                    min_kopt_o = perm
                    repeat = False
        if repeat:
            continue

        swaps = [(old, new) for old, new in zip(suborder, min_kopt_o)]
        G.reassign_positions(swaps)
        c = G.countcrossings_segtree()
        if c < min_crossings:
            min_crossings = c
            min_G = copy.deepcopy(G)
        else:
            G.reassign_positions([(x[1], x[0]) for x in swaps]) # swap them back

    assert min_G.countcrossings_segtree() == min_crossings
    return min_G, min_crossings


def k_opt_simulated_annealing(G, k, initial_temperature, cooling_rate):
    t = initial_temperature
    min_G = copy.deepcopy(G)
    min_crossings = G.countcrossings_segtree()
    
    while t > 1:
        # Get k random edges s.t. every edge crosses at least one of the other k-1 edges
        k_edges = [None for _ in range(k)]

        repeat = True
        s = time.time()
        while repeat and (time.time() - s < 1):
            k_edges[0] = random.sample(G.edgeset, k=1)[0]
            endpoints = set([k_edges[0][1]])
            for i in range(1, k):
                k_edges[i] = random.sample(G.edgeset, k=1)[0]
                while (not any(G.cross(*edge, *k_edges[i]) for edge in k_edges[:i]) and
                       time.time() - s < 1):
                    k_edges[i] = random.sample(G.edgeset, k=1)[0]
                endpoints.add(k_edges[i][1])
            if len(endpoints) < 2:
                continue
            suborder = list(endpoints)
            suborder.sort(key=lambda x: G.right_order[x])
            kosg = KOptSubGraph(k_edges)
            kosg.set_order(suborder)
            min_kopt_c = kosg.count_crossings()
            min_kopt_o = suborder
            for perm in permutations(suborder):
                kosg.set_order(perm)
                kopt_c = kosg.count_crossings()
                if kopt_c < min_kopt_c:
                    min_kopt_c = kopt_c
                    min_kopt_o = perm
                    repeat = False
        if repeat:
            t *= cooling_rate
            continue

        swaps = [(old, new) for old, new in zip(suborder, min_kopt_o)]
        G.reassign_positions(swaps)
        
        c = G.countcrossings_segtree()
        delta = c - min_crossings
        if accept(delta, t):
            min_crossings = c
            min_G = copy.deepcopy(G)
        else:
            G.reassign_positions([(x[1], x[0]) for x in swaps]) # swap them back
        
        t *= cooling_rate

    assert min_G.countcrossings_segtree() == min_crossings
    return min_G, min_crossings


def iterated_k_opt_simulated_annealing(G, n_iterations, k, initial_temperature, cooling_rate):
    min_G = copy.deepcopy(G)
    min_crossings = G.countcrossings_segtree()
    
    for _ in range(n_iterations):
        H = copy.deepcopy(min_G)
        H, c = k_opt_simulated_annealing(H, k, initial_temperature, cooling_rate)
        print(c)
        if c < min_crossings:
            min_crossings = c
            min_G = copy.deepcopy(H)

    assert min_G.countcrossings_segtree() == min_crossings
    return min_G, min_crossings

In [ ]:
G = load_graph(GRAPH)
G.barycenter()
G, c = k_opt_hill_climbing(G, 3, 500)
print(f"{c} Crossings.")

In [ ]:
G = load_graph(GRAPH)
G.barycenter()
G, c = k_opt_simulated_annealing(G, 3, 100, 0.9)
print(f"{c} Crossings.")

In [ ]:
G = load_graph(GRAPH)
G.barycenter()
H, c = iterated_k_opt_simulated_annealing(G, 20, 3, 100, 0.9)
print(f"Minimum: {c} Crossings.")